In [2]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

## Tables overview

In [2]:
%%sql
SELECT 
    name,
    type
FROM sqlite_master
WHERE type in ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


## 1.The most selling genres in USA

In [17]:
%%sql
--First subquery to connect tracks to customer data so we can isolate the US
WITH 
usa_tracks AS
(
    SELECT
    il.*
    FROM invoice_line il
    LEFT JOIN invoice i ON i.invoice_id = il.invoice_id
    LEFT JOIN customer c ON c.customer_id = i.customer_id
    WHERE c.country == "USA"
),
--Joining track table with the genre name
track_genre AS 
(
    SELECT
    t.*,
    g.name AS genre
    FROM track t
    LEFT JOIN genre g ON g.genre_id = t.genre_id
)

SELECT
tg.genre AS Genre,
SUM(ut.quantity) "Number of Tracks",
ROUND((CAST(SUM(ut.quantity) AS FLOAT) / (SELECT COUNT(*) FROM usa_tracks))*100, 1) || "%" AS "Genre proportion"
FROM usa_tracks AS ut
LEFT JOIN track_genre AS tg ON tg.track_id = ut.track_id
GROUP BY 1
ORDER BY 2 DESC;

Done.


Genre,Number of Tracks,Genre proportion
Rock,561,53.4%
Alternative & Punk,130,12.4%
Metal,124,11.8%
R&B/Soul,53,5.0%
Blues,36,3.4%
Alternative,35,3.3%
Latin,22,2.1%
Pop,22,2.1%
Hip Hop/Rap,20,1.9%
Jazz,14,1.3%


The final table says to us that the most likely genre in US are Rock(53%), Alternative&Punk(12,4%), Metal(11,8%)

## 2. Sales perfomance by sales agents
In this query I'll find the total dollar amount of sales assigned to each sales support agent within the company

In [34]:
%%sql
SELECT
e.first_name || " " || e.last_name AS Agent_name,
ROUND(SUM(i.total)) AS Total_sales,
e.title AS Title,
e.hire_date AS "Date of hiring"
FROM employee e
LEFT JOIN customer c ON c.support_rep_id = e.employee_id
LEFT JOIN invoice i ON i.customer_id = c.customer_id
WHERE title LIKE "%Support%"
GROUP BY c.support_rep_id
ORDER BY 2 DESC;

Done.


Agent_name,Total_sales,Title,Date of hiring
Jane Peacock,1732.0,Sales Support Agent,2017-04-01 00:00:00
Margaret Park,1584.0,Sales Support Agent,2017-05-03 00:00:00
Steve Johnson,1394.0,Sales Support Agent,2017-10-17 00:00:00


The most "useful" agent is Jane Peacock, She made a total amount 1731$, the 2nd 1584$ and 3rd 1394$

## 3. Sales by country

In the query we will find out for each country: 
- total number of customers
- total value of sales
- average value of sales per customer
- average order value

In [62]:
%%sql 
WITH countries AS
(
    SELECT CASE WHEN
    (
        SELECT COUNT(*) FROM customer
        WHERE c.country = country
    ) = 1 THEN "Other"
    ELSE c.country END "Country", c.customer_id, i.*
    FROM invoice i
    INNER JOIN  customer c  on i.customer_id = c.customer_id
)    

SELECT country, total_customers, total_sales, avg_sales_per_customer, avg_order_value
FROM (
    SELECT country, 
    COUNT(distinct customer_id) AS "total_customers",
    SUM(total)AS "total_sales", 
    SUM(total) / COUNT(distinct customer_id) AS "avg_sales_per_customer",
    SUM(total) / COUNT(distinct invoice_id) AS "avg_order_value",
    CASE
        WHEN country = "Other" THEN 1
        ELSE 0
        END AS sort
    FROM countries
    GROUP BY country
    ORDER BY sort, total_sales DESC
     
)

Done.


country,total_customers,total_sales,avg_sales_per_customer,avg_order_value
USA,13,1040.4899999999998,80.0376923076923,7.942671755725189
Canada,8,535.5900000000001,66.94875000000002,7.047236842105265
Brazil,5,427.68000000000006,85.53600000000002,7.0111475409836075
France,5,389.0699999999999,77.81399999999998,7.781399999999998
Germany,4,334.62,83.655,8.161463414634147
Czech Republic,2,273.24000000000007,136.62000000000003,9.108000000000002
United Kingdom,3,245.52,81.84,8.768571428571429
Portugal,2,185.13000000000002,92.56500000000001,6.383793103448276
India,2,183.14999999999998,91.57499999999999,8.72142857142857
Other,15,1094.9400000000003,72.99600000000002,7.44857142857143


USA has the biggest total number of customers, also the biggest total value of sales. In average customers from Czech spend more than everyone else and average order value as well

## 4.Categorization of invoices: Albums vs Individual tracks

In [71]:
%%sql
WITH category_invoice AS 
(
    SELECT t.track_id AS track_id, i.invoice_id
            FROM track t
            INNER JOIN invoice_line il ON il.track_id = t.track_id
            INNER JOIN invoice i ON i.invoice_id = il.invoice_id
            GROUP BY i.invoice_id
)
SELECT 
album_purchase,
COUNT(invoice_id) AS total_invoices,
ROUND(CAST(COUNT(invoice_id) AS FLOAT) / (SELECT COUNT(*) FROM invoice) * 100, 1) AS invoice_percentages
FROM 
(
    SELECT ci.*,
        CASE
            WHEN(
                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ci.track_id
                                     ) 

                  EXCEPT 

                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ci.invoice_id
                 ) IS NULL
             AND (
                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ci.invoice_id

                  EXCEPT 

                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ci.track_id
                                     ) 
                 ) IS NULL
             THEN "YES"
             ELSE "NO"
         END AS "album_purchase"
     FROM category_invoice ci
    )
GROUP BY album_purchase;

Done.


album_purchase,total_invoices,invoice_percentages
NO,500,81.4
YES,114,18.6


As a result people prefer buy tracks individually (81,4%) rather than the whole albums at once (18,6%)

## 5. The artist plaled in the most playlists

In [9]:
%%sql
SELECT ar.name AS artist_name, COUNT(p.playlist_id) AS number_of_appears, p.name AS playlist_name
FROM playlist_track AS pl
LEFT JOIN playlist p ON pl.playlist_id = p.playlist_id
LEFT JOIN track t ON pl.track_id = t.track_id
LEFT JOIN album al ON t.album_id = al.album_id
LEFT JOIN artist ar ON al.artist_id = ar.artist_id
GROUP BY 1
HAVING number_of_appears > 100
ORDER BY 2 DESC;

Done.


artist_name,number_of_appears,playlist_name
Iron Maiden,516,Heavy Metal Classic
U2,333,Music
Metallica,296,Heavy Metal Classic
Led Zeppelin,252,Music
Deep Purple,226,Music
Lost,184,TV Shows
Pearl Jam,177,Grunge
Eric Clapton,145,Brazilian Music
Faith No More,145,Music
Lenny Kravitz,143,Music


As a result Iron Maiden the most popular band that playlists usually contain. The 2nd place U2 with 333 appears and 3rd Metallica with 296

## 6. How many tracks have been purchased vs not purchased?

In [12]:
%%sql
WITH track_invoice AS 
(
    SELECT track_id FROM invoice_line
)
SELECT 
COUNT(DISTINCT ti.track_id) AS purchased_tracks,
(
    SELECT COUNT(DISTINCT track_id) FROM track WHERE track_id NOT IN 
    (SELECT track_id from track_invoice) 
)   AS not_purchased_tracks
FROM track_invoice ti
LEFT JOIN track t ON t.track_id = ti.track_id;

Done.


purchased_tracks,not_purchased_tracks
1806,1697


1806 tracks were purchased against 1697 were not. 

## 7. Is the range of tracks in the store reflective of their sales popularity?

In [22]:
%%sql
WITH range_tracks AS 
(SELECT il.*, t.genre_id
 FROM invoice_line AS il
 INNER JOIN track AS t ON t.track_id = il.track_id
)
SELECT 
g.name AS genre,
ROUND(SUM(rt.quantity) / (SELECT CAST(SUM(quantity) AS FLOAT) FROM invoice_line)*100,1) AS percentage_popularity,
ROUND(SUM(rt.unit_price * rt.quantity),1) AS invoice_amount
FROM range_tracks rt
INNER JOIN genre AS g ON g.genre_id = rt.genre_id

GROUP BY 1
ORDER BY 2 DESC,3 DESC;

Done.


genre,percentage_popularity,invoice_amount
Rock,55.4,2608.6
Metal,13.0,612.8
Alternative & Punk,10.3,487.1
Latin,3.5,165.3
R&B/Soul,3.3,157.4
Blues,2.6,122.8
Jazz,2.5,119.8
Alternative,2.5,115.8
Easy Listening,1.6,73.3
Pop,1.3,62.4


Obviously yes, Rock has 55% of the overall range of tracks and as the result the largest invoice amount. The next is metal with 13% and then Alternative & Punk with 10%.

## 8. protected vs non-protected media types? Effect on popularity

In [24]:
%%sql
SELECT name FROM media_type;

Done.


name
MPEG audio file
Protected AAC audio file
Protected MPEG-4 video file
Purchased AAC audio file
AAC audio file


In [31]:
%%sql 
WITH track_media AS 
( 
    SELECT 
        CASE 
        WHEN m.name LIKE "%Protected%"
        THEN "YES"
        ELSE "NO"
        END protected,
        t.track_id
    FROM media_type AS m
        INNER JOIN track t ON t.media_type_id = m.media_type_id
)
SELECT tm.protected,
COUNT(DISTINCT il.track_id) AS count_tracks,
ROUND(CAST(COUNT(DISTINCT il.track_id) AS FLOAT) / 1806 * 100, 1) AS percentage_tracks
FROM invoice_line AS il
LEFT JOIN track_media AS tm ON il.track_id = tm.track_id

GROUP BY 1;

Done.


protected,count_tracks,percentage_tracks
NO,1652,91.5
YES,154,8.5


most unprotected tracks among all, which allows us to say about their greater popularity